In [1]:
import pandas as pd

import sys

sys.path.append("./..")
from src.utils.get_youtube_url_official import find_urls_for_songs

In [2]:
def get_uri_for_curated(number_of_songs=4):
    df = pd.read_csv("../data/top_songs_curated.csv")
    df_missing = df[df["youtube_title"].isna()]

    df_missing_n = df_missing.head(number_of_songs)

    songs = (
        df_missing_n[["Track Name", "Artist Name(s)"]]
        .rename(columns={"Track Name": "title", "Artist Name(s)": "artist"})
        .to_dict(orient="records")
    )

    results = find_urls_for_songs(songs)
    results = pd.DataFrame.from_dict(results, orient="index")
    results = results[results["youtube_title"] != "Not found"]

    merged_df = df.merge(
        results,
        how="left",
        left_on=["Track Name", "Artist Name(s)"],
        right_on=["title", "artist"],
        suffixes=("", "_drop"),
    )

    merged_df["youtube_title"] = merged_df["youtube_title"].combine_first(
        merged_df["youtube_title_drop"]
    )
    merged_df["youtube_url"] = merged_df["youtube_url"].combine_first(
        merged_df["youtube_url_drop"]
    )
    merged_df = merged_df.drop(
        columns=["title", "artist", "youtube_title_drop", "youtube_url_drop"]
    )

    merged_df = merged_df.sort_values(["Artist Name(s)", "Track Name"])
    merged_df.to_csv("../data/top_songs_curated.csv", index=False)

In [3]:
get_uri_for_curated(number_of_songs=100)

API key loaded successfully!
Searching for 'The Hills' by The Weeknd...
Searching for 'Baba O'Riley' by The Who...
Searching for 'Happy Jack' by The Who...
Searching for 'I'm A Boy - Mono Version' by The Who...
Searching for 'My Generation - Mono Version' by The Who...
Searching for 'My Generation - Stereo Version' by The Who...
Searching for 'Pictures Of Lily' by The Who...
Searching for 'Pinball Wizard' by The Who...
Searching for 'Shakin' All Over - Live' by The Who...
Searching for 'Substitute' by The Who...
Searching for 'Who Are You' by The Who...
Searching for 'Won't Get Fooled Again - Original Album Version' by The Who...
Searching for 'The Best' by Tina Turner...
Searching for 'The Best - Edit' by Tina Turner...
Searching for 'We Don't Need Another Hero (Thunderdome)' by Tina Turner...
Searching for 'What You Get Is What You See' by Tina Turner...
Searching for 'What's Love Got to Do with It' by Tina Turner...
Searching for 'HIGHEST IN THE ROOM' by Travis Scott...
Searching fo